In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="",#Your model API key here
)
messages=[ {"role":"system","content":
       """
You are a helpful assistant. Solve this puzzle for me.
On a one-dimensional board, there are red checkers (’R’), blue checkers (’B’), and one empty space (’_’). A checker can move by either:
1. Sliding forward into an adjacent empty space, or
2. Jumping over exactly one checker of the opposite color to land in an empty space. The goal is to swap the positions of all red and blue checkers, effectively mirroring the initial state.
Example: If the initial state is [’R’, ’_’, ’B’], the goal is to reach [’B’, ’_’, ’R’]. Your solution should be a list of moves where each move is represented as [checker_color, position_from, position_to]. For example:
moves = [[ ’R ’ , 0 , 1] , [ ’B ’ , 2 , 0] , [ ’R ’, 1 , 2]]
This means: Move the red checker from position 0 to 1, then move the blue checker from position 2 to 0, and so on.
Requirements:
• When exploring potential solutions in your thinking process, always include the corresponding complete list of moves.
• The positions are 0-indexed (the leftmost position is 0).
• Ensure your final answer includes the complete list of moves for final solution in the format: moves = [[checker_color, position_from, position_to], ...]


       """
      },
      {"role": "user", "content":
       """
I have a puzzle with 7 positions, where 3 red checkers (’R’) on left, 3 blue checkers (’B’) on right, and one empty space (’_’) in between are arranged in a line.
Initial board: R R ... R _ B B ... B
Goal board: B B ... B _ R R ... R
Rules:
• A checker can slide into an adjacent empty space.
• A checker can jump over exactly one checker of the opposite color to land in an empty space.
• Checkers cannot move backwards (towards their starting side).
Find the minimum sequence of moves to transform the initial board into the goal board.

       """},
    ]

completion = client.chat.completions.create(
  extra_headers={
  },
  extra_body={"enable_thinking": True},
  model="qwen/qwen3-32b",
  max_tokens=64000,
  messages=messages
)
print(completion.choices[0].message.content)

In [ ]:
from typing import List, Tuple

Move = Tuple[str, int, int]   # ('R' | 'B', src_index, dst_index)

class CheckerJumpingValidator:
    """
    Validate a 1-D checker-jumping solution sequence.

    Parameters
    ----------
    n : int
        Number of red checkers on the left / blue checkers on the right.
        The board length is 2*n + 1 (one extra for the empty slot).
    """

    def __init__(self, n: int) -> None:
        if n <= 0:
            raise ValueError("n must be positive")
        self.n = n
        self.board = ['R'] * n + ['_'] + ['B'] * n
        self.goal  = ['B'] * n + ['_'] + ['R'] * n

    # ------------------------------------------------------------
    # public interface
    # ------------------------------------------------------------
    def validate(self, moves: List[Move]) -> Tuple[bool, str]:
        """
        Validate the entire sequence.

        Returns
        -------
        (ok: bool, message: str)
            ok == True  → 全部合法且到达终局
            ok == False → message 里指出第一处错误
        """
        for step, move in enumerate(moves, 1):
            ok, msg = self._apply_one_move(*move)
            if not ok:
                return False, f"Step {step}: {msg}"
        if self.board != self.goal:
            return False, "Board does not match goal configuration"
        return True, "OK"

    # ------------------------------------------------------------
    # helpers
    # ------------------------------------------------------------
    def _apply_one_move(self, colour: str, src: int, dst: int) -> Tuple[bool, str]:
        colour = colour.upper()
        if colour not in {"R", "B"}:
            return False, f"Illegal colour {colour!r}"

        if not (0 <= src < len(self.board)) or not (0 <= dst < len(self.board)):
            return False, "Source/Destination index out of range"

        if self.board[src] != colour:
            return False, f"Source {src} does not contain {colour}"

        if self.board[dst] != '_':
            return False, f"Destination {dst} is not empty"

        delta = dst - src

        # direction rule
        if colour == 'R' and delta <= 0:
            return False, "Red checker must move to the right"
        if colour == 'B' and delta >= 0:
            return False, "Blue checker must move to the left"

        # slide or jump
        if abs(delta) == 1:           # slide
            pass
        elif abs(delta) == 2:         # jump
            mid = (src + dst) // 2
            mid_piece = self.board[mid]
            if mid_piece == '_' or mid_piece == colour:
                return False, "Invalid jump: no opponent piece in the middle"
        else:
            return False, "Illegal move distance (must be 1 or 2)"

        # apply move
        self.board[dst], self.board[src] = self.board[src], '_'
        return True, "move OK"


# ------------------------------------------------------------
# example usage
# ------------------------------------------------------------
if __name__ == "__main__":
    # 一个合法的 n=5（左右各 5 棋）最短 35 步解；仅作演示
    moves = ['R', 2, 3],     # Move red at position 2 to 3
    ['B', 4, 2],     # Move blue at position 4 to 2 (jumping over red at 3)
    ['R', 3, 4],     # Move red at position 3 to 4
    ['B', 5, 3],     # Move blue at position 5 to 3 (jumping over red at 4)
    ['R', 1, 2],     # Move red at position 1 to 2 (jumping over blue at 2)
    ['B', 3, 1],     # Move blue at position 3 to 1 (jumping over red at 2)
    ['R', 0, 3],     # Move red at position 0 to 3 (jumping over blue at 1 and red at 2)
    ['B', 6, 4],     # Move blue at position 6 to 4 (jumping over red at 5)
    ['R', 3, 1],     # Move red at position 3 (now at 3) to 1 (jumping over blue at 1)
    ['B', 1, 3],     # Move blue at position 1 to 3
    ['R', 2, 1],     # Move red at position 2 to 1
    ['B', 4, 2],     # Move blue at position 4 to 2 (jumping over red at 2)
    ['R', 3, 4],     # Move red at position 3 to 4 (slide)
    ['B', 2, 3],     # Move blue at position 2 to 3
    ['R', 4, 5]      # Move red at position 4 to 5 (slide)
    validator = CheckerJumpingValidator(n=3)
    ok, message = validator.validate(moves)
    print("Result:", ok, "-", message)

Result: False - Board does not match goal configuration
